In [1]:
import common as ak
from operator_plan import OperatorPlan
from cost_packet import CostPacket
import numpy as np
import numpy.typing as npt

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US")
#char_dict = ak.get_char_dict(lang="en_US")
char_dict = ak.get_char_dict(lang="zh_CN")
module_dict = ak.get_module_dict(lang="zh_CN")
level_dict = ak.get_level_dict(lang="zh_CN")

In [3]:
n_items = len(item_dict.keys())
n_chars = len(char_dict.keys())
item_names, item_names_rev = ak.get_item_info(item_dict)
#char_names, char_names_rev = ak.get_char_info(char_dict)
char_names, char_names_rev = ak.get_char_translations(char_dict)

In [4]:
packet_all, _ = ak.get_all_char_all_costs(char_dict, module_dict, level_dict, n_chars)

In [20]:
plan_nearl = OperatorPlan(
    name = "Passenger",
    level_range = (1, 50),
    elite_range = (0, 2),
    skill_range = (1, 1),
    mastery_range = [
        (0, 0),
        (0, 0),
        (0, 0)
    ],
    module_range = [
        (0, 0),
        (0, 0)
    ]
)

mats_combined = plan_nearl.get_cost(char_names_rev, packet_all)
plan_nearl.pretty_print(item_names, mats_combined)

Passenger:
	E0 -> E2
Cost:
	Orirock Cube: 5
	Oriron: 8
	Oriron Block: 5
	Bipolar Nanoflake: 4
	Caster Chip: 5
	Caster Dualchip: 4
	LMD: 694154
	EXP: 512161


# Testing - Loading Gamepress from JSON

In [6]:
import json
with open("gamepress.json") as f:
    gpdata = json.load(f)["operators"]

In [7]:
n_gp_operators = len(gpdata)
gp_plans = np.empty(n_gp_operators, dtype=OperatorPlan)

In [8]:
for i, op in enumerate(gpdata):
    try:
        ## gamepress id doesnt match in-game id data
        name = char_names_rev[op["operator"]]
    except:
        name = op["operator"]
    #print(name)
    gp_plans[i] = OperatorPlan(
            name = name,
            elite_range = (op["start_promotion"], op["end_promotion"]),
            skill_range = (op["start_skill"], op["end_skill"]),
            mastery_range = np.transpose((op["start_specs"], op["end_specs"]))
        )

# Test - Serialising OperatorPlan objects

In [9]:
obj_str = plan_nearl.to_json()

new_dict = json.loads(obj_str)
new_plan = OperatorPlan(**new_dict)

print(new_plan)

# Test - User interface

In [10]:
preset_e1 = OperatorPlan(elite_range=(0,1))
preset_e1_SL7 = OperatorPlan(elite_range=(0,1), skill_range=(1,7))
preset_e2 = OperatorPlan(elite_range=(0,2))
preset_e2_SL7 = OperatorPlan(elite_range=(0,2), skill_range=(1,7))
preset_e2_S1M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,3),(0,0),(0,0)])
preset_e2_S2M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,3),(0,0)])
preset_e2_S3M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,0),(0,3)])
preset_S1M3 = OperatorPlan(mastery_range=[(0,3),(0,0),(0,0)])
preset_S2M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,0)])
preset_S3M3 = OperatorPlan(mastery_range=[(0,0),(0,0),(0,3)])
preset_S2M3_S3M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,3)])
preset_M9 = OperatorPlan(mastery_range=[(0,3),(0,3),(0,3)])
preset_M1L1 = OperatorPlan(module_range=[(0,1)])
preset_M1L3 = OperatorPlan(module_range=[(0,3)])

In [11]:
suzuran = preset_e1.as_operator("SilverAsh")
suzuran_total = suzuran.get_cost(char_names_rev, packet_all)
suzuran.pretty_print(item_names, suzuran_total)

SilverAsh:
	E0 -> E1
Cost:
	Polyester: 8
	Device: 3
	Guard Chip: 5
	LMD: 30000
	EXP: 24400


In [12]:
preset_bad = OperatorPlan(module_range=[(1, 1)])

In [13]:
print(item_names_rev["EXP"])

5001
